# Loading Packages

In [1]:
import Pkg
basedir = "../"
Pkg.activate(basedir)
using BenchmarkTools
using FreqTables
input = readlines(basedir*"data/input_07.txt")

  Activating project at `/uufs/chpc.utah.edu/common/home/sdss42/sdsswork/users/u6039752-1/software/AdventOfCode2023.jl`


1000-element Vector{String}:
 "53AQ3 698"
 "555A5 490"
 "KQQ8K 338"
 "Q8J88 94"
 "T844J 329"
 "99A9A 918"
 "5846Q 753"
 "99TJT 175"
 "A3465 353"
 "J9K99 725"
 "6A9Q8 525"
 "5Q2K3 666"
 "88688 729"
 ⋮
 "597J6 46"
 "TTT49 324"
 "2222A 273"
 "T5223 91"
 "97QQJ 812"
 "37373 776"
 "KKKQJ 327"
 "77K9J 280"
 "TT3T3 711"
 "92999 567"
 "99993 974"
 "52T22 735"

In [2]:
input_test = readlines(basedir*"data/input_07t.txt")

5-element Vector{String}:
 "32T3K 765"
 "T55J5 684"
 "KK677 28"
 "KTJJT 220"
 "QQQJA 483"

# Part 1

In [10]:
card2val = Dict('A'=>14,'K'=>13,'Q'=>12,'J'=>11,'T'=>10,'9'=>9,'8'=>8,'7'=>7,'6'=>6,'5'=>5,'4'=>4,'3'=>3,'2'=>2)

Dict{Char, Int64} with 13 entries:
  'K' => 13
  'J' => 11
  '7' => 7
  '6' => 6
  '9' => 9
  'T' => 10
  '5' => 5
  'A' => 14
  '3' => 3
  '2' => 2
  '4' => 4
  '8' => 8
  'Q' => 12

In [4]:
function ingest_input(input)
    sinput = split.(input," ")
    bid_vec = map(x->parse(Int,x[2]),sinput);
    hand_vec = zeros(Int,length(bid_vec),5);
    for i=1:length(bid_vec)
        hand_vec[i,:] .= map(x->card2val[x],collect(sinput[i][1]))
    end
    return bid_vec, hand_vec
end

ingest_input (generic function with 1 method)

In [5]:
function type_hand(hand)
    f = freqtable(hand)
    if maximum(f.array) == 5 # five of a kind
        return 7
    elseif maximum(f.array) == 4 # four of a kind
        return 6
    elseif (f.array == [2, 3]) .| (f.array == [3, 2]) # full house
        return 5
    elseif maximum(f.array) == 3 # three of a kind
        return 4
    elseif count(f.array .== 2) == 2 # two pair
        return 3
    elseif maximum(f.array) == 2 # pair
        return 2
    elseif length(f.array) == 5 # high card
        return 1
    end
end

type_hand (generic function with 1 method)

In [6]:
function solution(input)
    bid_vec, hand_vec = ingest_input(input)
    handtype_vec = map(x->type_hand(x),eachrow(hand_vec))
    
    hand_and_index = hcat(hand_vec,1:size(hand_vec,1))
    indvec = []
    for i=7:-1:1
        msk = (handtype_vec .== i)
        sort_mat = sortslices(hand_and_index[msk,:], dims=1, by=x->(x[1], x[2], x[3], x[4], x[5]),rev=true)
        push!(indvec,sort_mat[:,6]...)
    end
    return sum(bid_vec[indvec].*(size(hand_vec,1):-1:1))
end

solution (generic function with 1 method)

In [7]:
solution(input_test)

6440

In [8]:
solution(input)

246424613

In [11]:
@benchmark solution(input)

BenchmarkTools.Trial: 928 samples with 1 evaluation.
 Range (min … max):  4.980 ms … 10.068 ms  ┊ GC (min … max): 0.00% … 38.45%
 Time  (median):     5.032 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.389 ms ±  1.091 ms  ┊ GC (mean ± σ):  6.02% ± 11.65%

  █▅                                                      ▂   
  ███▅▄▁▆▅▄▃▁▁▃▁▃▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███ ▇
  4.98 ms      Histogram: log(frequency) by time      8.9 ms <

 Memory estimate: 3.97 MiB, allocs estimate: 54751.

# Part 2

In [12]:
card2val = Dict('A'=>14,'K'=>13,'Q'=>12,'J'=>0,'T'=>10,'9'=>9,'8'=>8,'7'=>7,'6'=>6,'5'=>5,'4'=>4,'3'=>3,'2'=>2)

Dict{Char, Int64} with 13 entries:
  'K' => 13
  'J' => 0
  '7' => 7
  '6' => 6
  '9' => 9
  'T' => 10
  '5' => 5
  'A' => 14
  '3' => 3
  '2' => 2
  '4' => 4
  '8' => 8
  'Q' => 12

In [29]:
function ingest_input(input)
    sinput = split.(input," ")
    bid_vec = map(x->parse(Int,x[2]),sinput);
    hand_vec = zeros(Int,length(bid_vec),5);
    for i=1:length(bid_vec)
        hand_vec[i,:] .= map(x->card2val[x],collect(sinput[i][1]))
    end
    return bid_vec, hand_vec
end

ingest_input (generic function with 1 method)

In [65]:
function get_replace_value(handJfree)
    if length(handJfree)==0
        return 14
    else
        f = freqtable(handJfree)
        return sort(f.dicts[1].keys[findall(f.array.==maximum(f.array))],rev=true)[1]
    end
end

get_replace_value (generic function with 1 method)

In [66]:
function type_hand(hand)
    mskJ = (hand.==0)
    rval = get_replace_value(hand[.!mskJ])
    thand = copy(hand)
    thand[mskJ].=rval
    g = freqtable(thand)
    if maximum(g.array) == 5 # five of a kind
        return 7
    elseif maximum(g.array) == 4 # four of a kind
        return 6
    elseif (g.array == [2, 3]) .| (g.array == [3, 2]) # full house
        return 5
    elseif maximum(g.array) == 3 # three of a kind
        return 4
    elseif count(g.array .== 2) == 2 # two pair
        return 3
    elseif maximum(g.array) == 2 # pair
        return 2
    elseif length(g.array) == 5 # high card
        return 1
    end
end

type_hand (generic function with 1 method)

In [67]:
function solution(input)
    bid_vec, hand_vec = ingest_input(input)
    handtype_vec = map(x->type_hand(x),eachrow(hand_vec))
    hand_and_index = hcat(hand_vec,1:size(hand_vec,1))
    indvec = []
    for i=7:-1:1
        msk = (handtype_vec .== i)
        sort_mat = sortslices(hand_and_index[msk,:], dims=1, by=x->(x[1], x[2], x[3], x[4], x[5]),rev=true)
        push!(indvec,sort_mat[:,6]...)
    end
    return sum(bid_vec[indvec].*(size(hand_vec,1):-1:1))
end

solution (generic function with 1 method)

In [71]:
solution(input_test)

5905

In [72]:
solution(input)

248256639

In [ ]:
248256639